In [18]:
L(0,9,8)

(#3) [0,9,8]

In [3]:
torch.__version__

'1.6.0'

In [16]:
a = torch.randn(3,3,requires_grad = True)
a.data

tensor([[-1.7796,  1.1781, -0.7734],
        [ 0.6397, -1.6298, -0.7041],
        [ 0.2721,  0.8817, -0.0292]])

In [29]:
a.data.add_(1,alpha=10)

tensor([[18.2204, 21.1781, 19.2266],
        [20.6397, 18.3702, 19.2959],
        [20.2721, 20.8817, 19.9708]])

In [30]:
# using typing

In [33]:
def type_ex(data:str)->str:
    return 'hi'+ ' '+ data

In [35]:
type_ex('john')

'hi john'

In [ ]:
#create type aliases

from collections.abc import Sequence
names = tuple(str,str)
key_vals = dict(str,list)
def

# optimizers

In [36]:
import torch
import torch.nn as nn
#from fastcore.foundation import *
from fastcore.all import *

In [12]:
def merge(*ds):
    "Merge all dictionaries in `ds`"
    return {k:v for d in ds if d is not None for k,v in d.items()}

In [93]:
#export
class _BaseOptimizer():
    "Common functionality between `Optimizer` and `OptimWrapper`"
    def all_params(self, n=slice(None), with_grad=False):
        res = L((p,pg,self.state[p],hyper) for pg,hyper in zip(self.param_lists[n],self.hypers[n]) for p in pg)
        #import pdb;pdb.set_trace()
        return L(o for o in res if hasattr(o[0], 'grad') and o[0].grad is not None) if with_grad else res

    def _set_require_grad(self, rg, p,pg,state,h): p.requires_grad_(rg or state.get('force_train', False))
    def freeze_to(self, n):
        self.frozen_idx = n if n >= 0 else len(self.param_lists) + n
        if self.frozen_idx >= len(self.param_lists):
            warn(f"Freezing {self.frozen_idx} groups; model has {len(self.param_lists)}; whole model is frozen.")
        for o in self.all_params(slice(n, None)): self._set_require_grad(True,  *o)
        for o in self.all_params(slice(None, n)): self._set_require_grad(False, *o)

    def freeze(self):
        assert(len(self.param_lists)>1)
        self.freeze_to(-1)

    def set_freeze(self, n, rg, ignore_force_train=False):
        for p in self.param_lists[n]: p.requires_grad_(rg or (state.get('force_train', False) and not ignore_force_train))

    def unfreeze(self): self.freeze_to(0)
    def set_hypers(self, **kwargs): L(kwargs.items()).starmap(self.set_hyper)
    def _set_hyper(self, k, v):
        for v_,h in zip(v, self.hypers): h[k] = v_

    def set_hyper(self, k, v):
        if isinstance(v, slice):
            if v.start: v = even_mults(v.start, v.stop, len(self.param_lists))
            else: v = [v.stop/10]*(len(self.param_lists)-1) + [v.stop]
        v = L(v, use_list=None)
        if len(v)==1: v = v*len(self.param_lists)
        assert len(v) == len(self.hypers), f"Trying to set {len(v)} values for {k} but there are {len(self.param_lists)} parameter groups."
        self._set_hyper(k, v)

    @property
    def param_groups(self): return [{**{'params': pg}, **hp} for pg,hp in zip(self.param_lists, self.hypers)]
    @param_groups.setter
    def param_groups(self, v):
        for pg,v_ in zip(self.param_lists,v): pg = v_['params']
        for hyper,v_ in zip(self.hypers,v):
            for k,t in v_.items():
                if k != 'params': hyper[k] = t
                    
#export
def _update(state, new=None):
    if new is None: return state
    if isinstance(new, dict): state.update(new)
    return state

# export
class Optimizer(_BaseOptimizer):
    "Base optimizer class for the fastai library, updating `params` with `cbs`"
    _keep_on_clear = ['force_train', 'do_wd']
    def __init__(self, params, cbs, train_bn=True, **defaults):
        params = L(params)
        self.cbs,self.state,self.train_bn = L(cbs),defaultdict(dict),train_bn
        #import pdb;pdb.set_trace()
        defaults = merge(*self.cbs.attrgot('defaults'), defaults)
        self.param_lists = L(L(p) for p in params) if isinstance(params[0], (L,list)) else L([params])
        self.hypers = L({} for _ in range_of(self.param_lists))
        self.set_hypers(**defaults)
        self.frozen_idx = 0

    def zero_grad(self):
        for p,*_ in self.all_params(with_grad=True):
            p.grad.detach_()
            p.grad.zero_()

    def step(self):
        for p,pg,state,hyper in self.all_params(with_grad=True):
            for cb in self.cbs: state = _update(state, cb(p, **{**state, **hyper}))
            self.state[p] = state

    def clear_state(self):
        for p,pg,state,hyper in self.all_params():
            self.state[p] = {k: state[k] for k in self._keep_on_clear if k in state}

    def state_dict(self):
        state = [self.state[p] for p,*_ in self.all_params()]
        return {'state': state, 'hypers': self.hypers}

    def load_state_dict(self, sd):
        assert len(sd["hypers"]) == len(self.param_lists)
        assert len(sd["state"])  == sum([len(pg) for pg in self.param_lists])
        self.hypers = sd['hypers']
        self.state = {p: s for p,s in zip(self.all_params().itemgot(0), sd['state'])}

In [43]:
ll = nn.Linear(3,4)

In [15]:
for param in ll.parameters():
    print(param.data.shape)

torch.Size([4, 3])
torch.Size([4])


In [47]:
from collections import defaultdict
op = Optimizer(ll.parameters(),[])

In [51]:
op1 = Optimizer([[0,9]],[])
test_eq(op1.param_lists,[[0,9]])

op2 = Optimizer([0,9],[])
op2.param_lists

(#1) [[0, 9]]

In [57]:
#hpyers copies the default's from each of the callbacks and replicates it across as many param_groups


def test_arg(p,**kwargs):
    return p
test_arg.defaults = {'lr': 100}

op = Optimizer([0,9],[test_arg])
op.hypers

(#1) [{'lr': 100}]

In [61]:
opt = Optimizer([[1,2],[3]], test_arg, lr=[0.1,0.2])
opt.hypers

(#2) [{'lr': 0.1},{'lr': 0.2}]

In [64]:
opt = Optimizer([[1,2],[3],[4]], test_arg, lr=slice(1e-4,1e-2))
opt.hypers,opt.param_groups

((#3) [{'lr': 0.0001},{'lr': 0.001},{'lr': 0.01}],
 [{'params': (#2) [1,2], 'lr': 0.0001},
  {'params': (#1) [3], 'lr': 0.001},
  {'params': (#1) [4], 'lr': 0.01}])

In [52]:
merge({'h':1},{'h':3})-> # merge dict overrides.

{'h': 3}

In [74]:
def sgd_step(p, lr, **kwargs):
    p.data.add_(p.grad.data, alpha=-lr)
def test_param(val,grad):
    res = torch.Tensor([val]).float()
    res.grad = torch.Tensor([grad]).float()
    return res
p = test_param(1.0,0.1)
sgd_step(p,lr=1.0)

In [76]:
p,p.grad

(tensor([0.9000]), tensor([0.1000]))

In [95]:
def tst_param(val, grad=None):
    "Create a tensor with `val` and a gradient of `grad` for testing"
    tensor = torch.Tensor
    res = tensor([val]).float()
    res.grad = tensor([val/10 if grad is None else grad]).float()
    return res

r = L.range(4)
def tst_params(): return r.map(tst_param)

params = tst_params()
opt = Optimizer(params, sgd_step, lr=0.1)


In [97]:
opt.state_dict()

{'state': [{}, {}, {}, {}], 'hypers': (#1) [{'lr': 0.1}]}

In [ ]:
opt.step()
[p.item() for p in params]

In [103]:
#what the f is attrgot

def f_attrgot():
    get_me = 'tom'
    return 9
    
l_f = L(f_attrgot)

In [104]:
l_f.attrgot('get_me')

(#1) [None]

TypeError: attrgot() missing 1 required positional argument: 'k'

# Understanding pytorch